<div class="alert alert-warning">

Warning:

This tutorial is currently under construction!

</div>

# Tutorial: Container-based Inference Pipeline

The deployment of the auto-segmentation solution demonstrated in the ["DL-Model Inference Pipeline"](examples.inference.basic.ipynb) example is shown in this tutorial. Typically, the deployment of auto-segmentation solutions is a challenging task due to various reasons such as that the physician's PC does not have a [GPU](https://en.wikipedia.org/wiki/Graphics_processing_unit), Python can not be installed onto the target machine because access is restricted, no access to the machines in clinics for installation, and many more. In order to circumvent those challenges, the usage of [Docker](https://www.docker.com/) containers is a nice solution. The advantage of using Docker is manifold and include that the Docker image can be build by the developer without requiring access to the target machine, and that versioning of the Docker images is a built-in feature that comes at no costs. For more details on Docker we refer to the [official Docker documentation](https://docs.docker.com/).

This tutorial is not executable but explains how to build an auto-segmentation Docker image from a PyRaDiSe-based auto-segmentation solution. The aim of this tutorial is to facilitate dockerization of auto-segmentation algorithms such that these can be distributed easily.

## Workflow Overview
The following workflow provides an overview over the coarse steps that need to be taken to develop a Docker-based auto-segmentation solution:

1. [Develop Standalone Auto-segmentation Solution](#Develop-Standalone-Auto-segmentation-Solution)
2. [Install Docker with GPU Support](#Install-Docker-with-GPU-Support)
3. [Implement Communication API](#Implement-Communication-API)
4. [Dockerize the Project](#Dockerize-the-Project)
5. [Test Image Locally](#Test-Image-Locally)
6. [Store Image in Registry](#Store-Image-in-Registry)
7. [Automate Deployment Pipeline](#Automate-Deployment-Pipeline)
8. [Deploy to Target Machine](#Deploy-to-Target-Machine)


## Steps
The following chapters explain each step to take towards a dockerized auto-segmentation solution.

### Develop Standalone Auto-segmentation Solution
In general, one should start the development on a local machine without considering Docker too much.
The goal of this first step should be to have a robust auto-segmentation solution that implements all required functionality.
Furthermore, the project should have a proper structure what will make dockerization straightforward later.




In general, one should start developing an auto-segmentation solution standalone because the failure-free functionality should have the first priority.
During standalone development, one should focus on having a proper project structure with

- proper python structure
- requirements file
- use virtual environments

### Install Docker with GPU Support
- check for each OS
- regard licensing of Docker desktop
- how to enable gpu support

### Implement Communication API
- select a strategy, maybe given
- implement and test (using e.g. postman)
- connect the communication with the inferer

### Dockerize the Project
- provide resources
- how to write a dockerfile
- images to take with python installation
- vulnerability scanning

### Test Image Locally
- test the image
- use different data
- try to produce errors

### Store Image in Registry
- what are registries
- how to use them
- resources

### Automate Deployment Pipeline
- what tools to use
- how to achieve

### Deploy to Target Machine
- asdasd